# 第12章 TensorFlow计算加速

- 为了加速训练过程，本章将介绍如何通过TensorFlow利用GPU或/和分布式计算进行模型训练

## 12.1 TensorFlow使用GPU

- TensorFlow程序可以通过tf.device函数来指定运行每一个操作的设备，这个设备可以是本地的CPU或者GPU，也可以是某一台远程的服务器，但在本节中只关心本地的设备
- TensorFlow会给每一个可用的设备一个名称，tf.device函数可以通过设备的名称来指定执行运算的设备；比如CPU在TensorFlow中的名称为/cpu:0，在默认情况下，即使机器有多个CPU，TensorFlow也不会区分它们，所有的CPU都使用/cpu:0作为名称；而一台机器上不同GPU的名称是不同的，第n个GPU在TensorFlow中的名称为/gpu:n-1
- TensorFlow提供了一个快捷的方式来查看运行每一个运算的设备，在生成会话时，可以通过设置log_device_placement参数来打印运行每一个运算的设备

In [1]:
import tensorflow as tf

a = tf.constant([1.0, 2.0, 3.0], shape=[3], name='a')
b = tf.constant([1.0, 2.0, 3.0], shape=[3], name='b')
c = a + b
# 通过log_device_placement参数来输出运行每一个运算的设备
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print sess.run(c)

[2. 4. 6.]


- 在配置好GPU环境的TensorFlow中，如果操作没有明确地指定运行设备，那么TensorFlow会优先选择GPU，如果有多个GPU，TensorFlow只会将运算优先放到/gpu:0上；如果需要将某些运算放到不同的GPU或者CPU上，就需要通过tf.device来手工指定

In [2]:
import tensorflow as tf

# 通过tf.device将运算指定到特定的设备上
with tf.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0], shape=[3], name='a')
    b = tf.constant([1.0, 2.0, 3.0], shape=[3], name='b')
    
with tf.device('/gpu:1'):
    c = a + b
    
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print sess.run(c)

InvalidArgumentError: Cannot assign a device for operation add_1: node add_1 (defined at <ipython-input-2-589869cd5098>:9) was explicitly assigned to /device:GPU:1 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device. The requested device appears to be a GPU, but CUDA is not enabled.
	 [[node add_1 (defined at <ipython-input-2-589869cd5098>:9) ]]

Caused by op u'add_1', defined at:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Library/Python/2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Python/2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    a

- 在TensorFlow中，不是所有的操作都可以被放在GPU上，如果强行将无法放在GPU上的操作指定到GPU上，那么程序会报错
- 不同版本的TensorFlow对GPU的支持不一样，如果程序中全部使用强制指定设备的方式会降低程序的可移植性，为避免这个问题，TensorFlow在生成会话时可以指定allow_soft_placement参数，当其设置为True时，如果运算无法由GPU执行，那么TensorFlow会自动将它放到CPU上执行
- 虽然GPU可以加速TensorFlow的计算，但一般来说不会把所有的操作全部放在GPU上，一个比较好的实践是将计算密集型的运算放在GPU上，而把其它操作放到CPU上

- TensorFlow默认会占用设备上的所有GPU以及每个GPU的所有显存，如果在一个TensorFlow程序中只需要使用部分GPU，可以通过设置CUDA_VISIBLE_DEVICES环境变量来控制
- 虽然TensorFlow默认会一次性占用一个GPU的所有显存，但是TensorFlow也支持动态分配GPU的显存，使得一块GPU上可以同时运行多个任务

## 12.2 深度学习训练并行模式

- TensorFlow可以很容易地利用单个GPU加速深度学习模型的训练过程，但要利用更多的GPU或者机器，需要了解如何并行化地训练深度学习模型，常用的并行化深度学习模型训练方式有两种，同步模式和异步模式
- 可以简单地认为异步模式就是单机模式复制了多份，每一份使用不同的训练数据进行训练，在异步模式下，不同设备之间是完全独立的；使用异步模式训练的深度学习模型有可能无法达到较优的训练结果
- 为了避免更新不同步的问题，可以使用同步模式，在同步模式下，所有的设备同时读取参数的取值，并且当反向传播算法完成之后同步更新参数的取值，单个设备不会单独对参数进行更新，而会等待所有设备都完成反向传播之后再统一更新参数
- 同步模式解决了异步模式中存在的参数更新问题，然而同步模式的效率却低于异步模式；在同步模式下，每一轮迭代都需要设备统一开始、统一结束，如果设备的运行速度不一致，那么每一轮训练都需要等待最慢的设备结束才能开始更新参数，于是很多时间都花在等待上；虽然理论上异步模式存在缺陷，但因为训练深度学习模型时使用的随机梯度下降本身就是梯度下降的一个近似解法，而且即使是梯度下降也无法保证达到全局最优值，所以在实际应用中，在相同时间内，使用异步模式训练的模型不一定比同步模式差；所以这两种训练模式在实践中都有非常广泛的应用

## 12.3 多GPU并行

- 一般来说一台机器上的多个GPU性能相似，所以在这种设置下会更多地采用同步模式训练深度学习模型

## 12.4 分布式TensorFlow

- 通过多GPU并行的方式可以达到很好的加速效果，然而一台机器上能够安装的GPU有限，要进一步提升深度学习模型的训练速度，就需要将TensorFlow分布式运行在多台机器上